# Estimating samples per SMRT-cell

Library > sample > reaction 

Individual IVT reactions will compose samples. In general reactions that belong to the same sample will include all inserts from a specific series of plasmids. Samples should include a transcribed untreated, transcribed RnaseH treated and untranscribed reactions.

In [12]:
reads_per_SMRT_cell = 4e6 * 0.60
per_reaction = reads_per_SMRT_cell / 3  # assuming transcribed, transcribed + RnaseH, and untranscribed
per_insert = per_reaction / 31
per_insert

25806.451612903227

Above would be reads per insert for 1 standard reaction assuming only get 60% of the SequelII advertised 4 million reads per SMRTcell. I think 1000-2000 reads per insert would be enough to do stats on so with that in mind.

In [13]:
per_insert / 2500

10.32258064516129

So could likely pull off around 10 *samples* (or 30 reactions) per SMRT cell. As long as there was a way to individualy distinguish reactions from each other.

One possible approach to doing this could be to amplify bisulfite treated products with overhanging PCR primers. 
The overhang would contain a unique ID for that reaction plus some buffer sequence on the 5' end to protect it from anything that might happen during library prep. This way after amplification any group of reactions could be combined into the same tube and could later be distinguised by sequencing.

## Designing overhanging barcoded primers

In [33]:
from dnachisel import *
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

In [40]:
class BarcodedPrimer():
    
    buffer_seq = None
    buffer_length = 15
    overhang_seqs = []
    
    def __init__(self, name, target, overhang_length=8, description=''):
        self.name = name
        self.target = target
        self.overhang_length = overhang_length
        self.overhang_seq = None
        self.description = description
    
    @classmethod
    def optimize_buffer(cls):
         cls.buffer_seq = cls._optimize_seq(cls.buffer_length)
        
    @classmethod
    def _optimize_seq(cls, length):
        
        def resolve_sequence():
            problem = DnaOptimizationProblem(
                sequence=random_dna_sequence(length),
                constraints=[
                    AvoidPattern("EcoRI_site"),
                    AvoidPattern("SacI_site"),
                    AvoidPattern("KpnI_site"),
                    AvoidPattern("HindIII_site"),
                    AvoidPattern('6xA'),  # A homopolymers
                    AvoidPattern('3x2mer'),  # Avoid direct repeats
                    EnforceGCContent(mini=0.4, maxi=0.6, window=length)

                ]           
            )
            problem.resolve_constraints()
            problem.optimize()
            return problem.sequence
        
        def hamming_dist(seq_a, seq_b):
            assert len(seq_a) == len(seq_b)
            return sum([1 for i in range(len(seq_a)) if seq_a[i] == seq_b[i]])
        
        # Generate sequences that fit required parameters. Reattempt
        # if the hamming distance between the generated overhang
        # and any previously generated ones is > 50%. This is to
        # ensure sequences are easily IDed upon sequencing
        while True:
            seq = resolve_sequence()
            for each_overhang in cls.overhang_seqs:
                if hamming_dist(seq, each_overhang) < int(length/2):
                    break
            break
        
        return seq
    
    
    def optimize_overhang(self):
        overhang = self._optimize_seq(self.overhang_length)
        attempts = 0
        while overhang in BarcodedPrimer.overhang_seqs:
            overhang = self._optimize_seq(self.overhang_length)
            attempts += 1
            if attempts > 4:
                raise Exception('Too many attempts!')
        self.overhang_seq = overhang
    
    
    def to_record(self):
        if self.overhang_seq == None:
            self.optimize_overhang()
        if BarcodedPrimer.buffer_seq == None:
            BarcodedPrimer.optimize_buffer()
        complete_seq = BarcodedPrimer.buffer_seq + self.overhang_seq + self.target
        
        return SeqRecord(
            Seq(complete_seq),
            id=self.name,
            name='',
            description=self.description
        )

b = BarcodedPrimer('test-1', '')
c = BarcodedPrimer('test-2', 'Target2')
print(b.to_record())
print(b.overhang_seq)
print(c.to_record())
print(c.overhang_seq)

location:   0%|                                  | 0/1 [00:00<?, ?it/s, now=0-8]


ID: test-1
Number of features: 0
Seq('GGGTGCAGGTAGGTAGTAGGACATarget1')
GTAGGACA


location:   0%|                                  | 0/1 [00:00<?, ?it/s, now=0-8]
                                                                                

ID: test-2
Number of features: 0
Seq('GGGTGCAGGTAGGTATCTGGTTCTarget2')
TCTGGTTC


### T7 init barcoded primers

First I used primer3 to pick a fwd primer in the region between ~ -150 bp and -250 bp from pFC9.
The complete sequence of this region is shown below.

```
tcgaggtgccgtaaagcactaaatcggaaccctaaagggagcccccgatttagagcttgacggggaaagccggcgaacgtggcgagaaaggaagggaagaaagcgaaaggagcgggcgctagggcgctggcaagtgtagcggtcacgctg
```

Primer3 picked out the following sequence using the web interface. Used default parameters except only picked a left primer.

```
agggaagaaagcgaaaggag
```

```
No mispriming library specified
Using 1-based sequence positions
OLIGO            start  len      tm     gc%   any    3' seq 
LEFT_PRIMER         93   20   59.96   50.00  2.00  0.00 agggaagaaagcgaaaggag
SEQUENCE SIZE: 150
INCLUDED REGION SIZE: 150        49   20   59.82   45.00  4.00  0.00 tttagagcttgacggggaaa
```

This is the primer I will use for the target in barcoded primers. Only one primer will need a barcode and so I selected fwd primer as the barcoded primer.

For the reverse primer I selected the region between 2000 and 2200 bp downstream of the pFC9VR1 construct. These region has the sequence below.

```
tccgcctccatccagtctattaattgttgccgggaagctagagtaagtagttcgccagttaatagtttgcgcaacgttgttgccattgctacaggcatcgtggtgtcacgctcgtcgtttggtatggcttcattcagctccggttcccaacgatcaaggcgagttacatgatcccccatgttgtgaaaaaaagcggttag
```

I then used the primer3 web interface to search for reverse primers in this region. Primer3 select this oligo as the best primer.

```
aagccataccaaacgacgag
```

```
No mispriming library specified
Using 1-based sequence positions
OLIGO            start  len      tm     gc%   any    3' seq 
RIGHT_PRIMER       130   20   60.13   50.00  3.00  0.00 aagccataccaaacgacgag
SEQUENCE SIZE: 200
INCLUDED REGION SIZE: 200
```

In all cases primer3 was used `primer3 release 1.1.4` was the version used.